In [1]:
from pyspark.sql import SparkSession


In [2]:
spark=SparkSession.builder.appName('recommend').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
input_data=spark.read.csv('movielens_ratings.csv',header=True,inferSchema=True)

In [6]:
input_data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [7]:
input_data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
train,test=input_data.randomSplit([0.8,0.2])

In [10]:
als=ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [11]:
model=als.fit(train)

In [12]:
predictions=model.transform(test)

In [13]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|     5|-0.6584447|
|     31|   1.0|    24| 1.3428924|
|     31|   1.0|    18| 1.2272226|
|     85|   3.0|     1|  1.318609|
|     85|   5.0|     8| 4.1039257|
|     85|   1.0|    23|-1.5043002|
|     65|   1.0|    22| 2.0473073|
|     65|   2.0|     3|  3.024958|
|     65|   2.0|     5| 2.4446163|
|     65|   1.0|     2| 1.9355017|
|     53|   1.0|    12| 0.6752472|
|     53|   2.0|    19| 0.8591999|
|     53|   5.0|    21|0.46956855|
|     78|   1.0|    19|0.15269554|
|     34|   1.0|    28| 3.3416567|
|     28|   1.0|    27| 0.4361066|
|     28|   3.0|     1| 1.5832604|
|     28|   1.0|     6|0.22993025|
|     76|   1.0|    26| 3.6548157|
|     76|   1.0|    25| 3.3608952|
+-------+------+------+----------+
only showing top 20 rows



In [14]:
evaluator=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [16]:
rmse=evaluator.evaluate(predictions)

In [17]:
rmse

1.730511240171716

#  Evaluating on new user

In [18]:
new_user=test.filter(test['userId']==11).select(['movieId','userId'])

In [20]:
new_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     45|    11|
|     47|    11|
|     59|    11|
|     66|    11|
|     70|    11|
|     76|    11|
|     77|    11|
|     86|    11|
|     94|    11|
|     99|    11|
+-------+------+



In [21]:
recommendations=model.transform(new_user)

In [22]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     77|    11|   2.6491709|
|     59|    11|   2.4110236|
|     66|    11|   2.2055058|
|     47|    11|    2.016239|
|     45|    11|   1.6288142|
|     94|    11|   1.6196722|
|     70|    11|   0.9808644|
|     86|    11|   0.7837828|
|     11|    11|-0.040923037|
|     99|    11| -0.32990766|
|     76|    11| -0.93029356|
+-------+------+------------+

